In [146]:
import pandas as pd
import datetime
import re
import numpy as np


data = pd.read_excel('Ошибки ПППК на 09.11.xlsx', dtype={'№ а/к':str})
spu = pd.read_excel('СПУ_2.xlsx', dtype={'Номер карты':str})  #В Клик ставим фильтр выбрать всех клиентов с СПУ в карточке

In [147]:
data['Нет СПУ в базе'] = 'Есть'
data['Нет СПУ в базе'][data[~data['№ а/к'].isin(spu['Номер карты'])].index] = 'Нет'  # поставим всем клиентам без СПУ в базе НЕТ 

C:\Users\AmanovRA\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [148]:
spu = spu[spu['Номер карты'].isin(data['№ а/к'])]

In [149]:
spu = spu.sort_values(by='Номер карты').reset_index(drop=True)

In [150]:
cur = datetime.datetime.now().date()

def check_dates(x):
    if isinstance(x, str):
        return True
    if x.date() >= cur:
        return True
    return False

spu['Дата оконч проверить'] = spu['Дата окончания СПУ из карточки Пациента'].apply(check_dates)

In [151]:
spu = spu[spu['Дата оконч проверить'] == True].reset_index(drop=True)

In [152]:
spu = spu.drop_duplicates(subset=['Номер карты', 'СПУ из карточки Пациента'], ignore_index=True)

In [153]:
group_spu = spu.groupby('Номер карты', as_index=False).agg(list)[['Номер карты', 'СПУ из карточки Пациента', 'Дата оконч проверить']]

In [154]:
data = data.merge(group_spu, left_on='№ а/к', right_on='Номер карты', how='left')

In [155]:
def get_percent(x):
    if x is np.NaN:
        return 'Нет действующей СПУ в базе'
    pattern = r'(\d{1,2}) ?%'
    arr = []
    for element in x:
        if re.search(pattern, element):
            arr.append(re.search(pattern, element).group(1))
        else:
            arr.append(element)
    if len(arr) == 1:
        if arr[0].isdigit():
            return int(arr[0])
        else:
            return 'Ручная проверка'
    else:
        return 'Несколько СПУ'
    

In [156]:
data['СПУ в процентах'] = data['СПУ из карточки Пациента'].apply(get_percent)

In [157]:
def chech_spu(row):
    if row['скидка'] == row['СПУ в процентах']:   # скдика как число в файле от Табенской Ксении
        return 'Верно'
    elif isinstance(row['СПУ в процентах'], int):
        return 'Неверный процент'
    else:
        return row['СПУ в процентах']

In [158]:
data['Результат проверки'] = data.apply(chech_spu, axis=1)

In [159]:
data.drop(['Дата оконч проверить', 'СПУ в процентах', 'Нет СПУ в базе', 'Номер карты'], axis=1, inplace=True)

In [160]:
data.to_excel('Проверка СПУ.xlsx', index=False)